# Import libraries

In [1]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tpot import TPOTClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import Normalizer
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn.metrics import recall_score, confusion_matrix, classification_report

C:\Users\Adil\anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


# Data Fetching

In [2]:
file_path= os.path.join("../data/data_bankruptcy.csv") # Filepath of CSV file

In [3]:
df = pd.read_csv(file_path)
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


# Data Cleaning

In [4]:
# drop the space before title of columns
df.columns = [c.replace(" ","", 1) if c.startswith(' ') else c for c in df.columns]

In [5]:
# drop 'Net Income Flag' feature
df = df.drop('Net Income Flag', axis=1)

# Feature Selections

In [6]:
X = df.iloc[:, 1:]  # List of features which are required for model training
y = df['Bankrupt?']  # Target feature for prediction.

# ML Modeling

## TPOT Classifier

### Data Splitting

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

### Preprocessing pipeline

In [8]:
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5, periodic_checkpoint_folder="./topt_checkpoints", verbosity=2, scoring='recall')
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9745967741935484

Generation 2 - Current best internal CV score: 0.9808467741935484

Generation 3 - Current best internal CV score: 0.9808467741935484

Generation 4 - Current best internal CV score: 0.9872983870967742

Generation 5 - Current best internal CV score: 0.9872983870967742

Best pipeline: GaussianNB(Normalizer(RFE(input_matrix, criterion=entropy, max_features=0.3, n_estimators=100, step=0.8), norm=max))


TPOTClassifier(generations=5,
               periodic_checkpoint_folder='../topt_checkpoints/tpot_model_1.txt',
               population_size=20, scoring='recall', verbosity=2)

### Model export

In [11]:
pipeline_optimizer.export('../models/tpot_company_bankruptcy.py')

### Model instantiation

In [ ]:
model_tpot = Pipeline(steps=[
    RFE(estimator=(ExtraTreesClassifier(criterion="entropy", max_features=0.3, n_estimators=100)), step=0.8),
    Normalizer(norm="max"),
    GaussianNB()]
)

### Model performance

In [8]:
# Get train scores, train sizes, and validation scores using `learning_curve`, recall
def learning_curves(model, features, target):
    train_sizes, train_scores, test_scores = learning_curve(estimator = model,
                                                            X = features,
                                                            y = target,
                                                            train_sizes = [5,10,50,100,200,500,1000,2000,3000,5000],
                                                            cv = 5,
                                                            scoring='recall',
                                                            shuffle = True,
                                                            random_state=3)

    # Take the mean of cross-validated train scores and validation scores
    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)

    # Plot the learning curves!
    plt.plot(train_sizes, train_scores_mean, label = 'Training score')
    plt.plot(train_sizes, test_scores_mean, label = 'Test score')
    plt.ylabel('Recall', fontsize = 14)
    plt.xlabel('Training set size', fontsize = 14)
    plt.title('Learning curves - log model', fontsize = 18, y = 1.03)
    plt.legend()
    plt.show()


In [9]:
# ROC Curve
def roc_auc(model, X_tst, y_tst):
    plot_roc_curve(model, X_tst, y_tst)
    plt.title('ROC Curve')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.show()


In [10]:
# Confusion Matrix
def plot_confusion_matrix(y_tst, y_hat):
    rec = round(recall_score(y_tst, y_hat), 2)
    cm = confusion_matrix(y_tst, y_hat)
    sns.heatmap(cm, annot=True, fmt=".0f")
    plt.xlabel('y_pred')
    plt.ylabel('y')
    plt.title('Recall Score: {0}'.format(rec), size=20)
    plt.show()


----------------------------